In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.contrib.training import bucket_by_sequence_length

tf.reset_default_graph()
BASE = '/home/brandon/terabyte/Datasets/test_data/'
file = os.path.join(BASE, 'train_from.txt.ids1000')
sess = None

omg this works


A **TFRecords** file contains a sequence of strings with CRC hashes. Each record has the format

```python
uint64 length
uint32 masked_crc32_of_length
byte   data[length]
uint32 masked_crc32_of_data
```

In [15]:
tfrecords_fname = 'test.tfrecords'
print("Currently in dir: ", os.getcwd())
train_file = os.path.join(BASE, 'train_from.txt.ids1000')
with tf.gfile.GFile(train_file, mode="r") as source_file:
    with tf.python_io.TFRecordWriter(tfrecords_fname) as writer:
        source = source_file.readline()
        while source:
            example = tf.train.SequenceExample()
            example.context.feature['length'].int64_list.value.append(len([int(x) for x in source.split()]))
            for x in source.split():
                example.feature_lists.feature_list['encoder_tokens'].feature.add().int64_list.value.append(int(x))
            writer.write(example.SerializeToString())
            #writer.write(tf.compat.as_bytes(source))
            source = source_file.readline()

Currently in dir:  /home/brandon/Documents/seq2seq_projects/notebooks


In [40]:
bucket_by_sequence_length??

In [5]:
sess = None
tf.reset_default_graph()
sess = None

In [2]:
capacity = 5
LAST_RUNNER = 'batch_dequeue'
CONTEXT_FEATURES = {'length': tf.FixedLenFeature([], dtype=tf.int64)}
SEQUENCE_FEATURES = {'encoder_tokens': tf.FixedLenSequenceFeature([], dtype=tf.int64)}
tfrecords_fname = 'test.tfrecords'

filename_queue = tf.train.string_input_producer([tfrecords_fname])
reader = tf.TFRecordReader(name='tfrecord_reader')
_, next_raw = reader.read(filename_queue, name='read_records')

q = tf.RandomShuffleQueue(capacity=capacity,min_after_dequeue=0,dtypes=tf.string,shapes=[()],name='randomize_records')
enqueue_op = q.enqueue(next_raw)
example_dq = q.dequeue()
seq_len_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=example_dq,
                                                     context_features=CONTEXT_FEATURES,
                                                     sequence_features=SEQUENCE_FEATURES)
seq_len = seq_len_parsed['length'] 
sequence = sequence_parsed['encoder_tokens']

numberOfThreads = 1
qr = tf.train.QueueRunner(q, [enqueue_op] * numberOfThreads)
# Don't forget to add your "QueueRunner" to the QUEUE_RUNNERS collection
tf.train.add_queue_runner(qr)

In [3]:
sequence_length, outputs = bucket_by_sequence_length(
    input_length=tf.to_int32(seq_len),
    tensors=[sequence],
    batch_size=2, bucket_boundaries=[2, 5], capacity=5,
    dynamic_pad=True, name='bucket_by_sequence_length', 
)

In [4]:
with tf.Session() as sess:
    i = 0
    try:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        while True:
            i += 1
            if i > 50: break
            fetched = sess.run({'out_len': sequence_length, 'out': outputs})
            print(fetched['out_len'])
            print(fetched['out'])
    except tf.errors.OutOfRangeError:
        coord.request_stop()
        coord.join(threads)

[2 4]
[array([[29,  5,  0,  0],
       [42, 32, 37,  5]])]
[9 6]
[array([[10,  7,  6, 14, 23,  6, 31, 19,  4],
       [40, 44,  9, 15, 45,  5,  0,  0,  0]])]
[6 5]
[array([[17,  8, 22, 43, 28,  4],
       [11,  7,  9, 27,  4,  0]])]
[9 5]
[array([[13, 30, 26, 12, 35, 41,  6, 39,  4],
       [10, 49, 18, 52,  4,  0,  0,  0,  0]])]
[6 5]
[array([[11, 20,  8, 24, 16,  4],
       [34, 46, 18, 21,  4,  0]])]
[4 2]
[array([[13,  8, 12,  4],
       [29,  5,  0,  0]])]
[6 5]
[array([[36, 47, 38, 25, 33,  5],
       [17, 48, 16, 51,  5,  0]])]
[5 9]
[array([[17, 48, 16, 50,  5,  0,  0,  0,  0],
       [10,  7,  6, 14, 23,  6, 31, 19,  4]])]
[6 6]
[array([[17,  8, 22, 43, 28,  4],
       [10,  7,  9, 15, 14,  4]])]
[6 6]
[array([[40, 44,  9, 15, 45,  5],
       [10,  7,  9, 15, 14,  4]])]
[4 4]
[array([[42, 32, 37,  5],
       [13,  8, 12,  4]])]
[5 5]
[array([[11,  7,  9, 27,  4],
       [34, 46, 18, 21,  4]])]
[6 6]
[array([[36, 47, 38, 25, 33,  5],
       [17,  8, 22, 43, 28,  4]])]
[5 9]
[ar